In [1]:
import json
import re

def extract_json_from_string(input_string):
    """
    Extract JSON-like content from a string, handling extra or mismatched braces, 
    and parse it into a Python dictionary.

    Args:
    input_string (str): The string containing JSON-like content.

    Returns:
    dict: Parsed JSON content as a dictionary, or an empty dictionary if parsing fails.
    """
    try:
        # Use regex to find JSON-like content
        json_pattern = re.compile(r'{.*?}', re.DOTALL)
        match = json_pattern.search(input_string)

        if match:
            # Handle potential double braces
            json_content = match.group()
            json_content = json_content.replace('{{', '{').replace('}}', '}')
            
            # Try parsing the adjusted JSON content
            return json.loads(json_content)
    except json.JSONDecodeError:
        pass

    # Return empty dictionary if no valid JSON is found
    return {}

# Example usage
input_string_1 = """
Some random text
{
    "issue_type": "other"
}
More text here
"""

input_string_2 = """
Some text with extra braces {{
    "issue_type": "error"
}}
End of text
"""

parsed_json_1 = extract_json_from_string(input_string_1)
parsed_json_2 = extract_json_from_string(input_string_2)

parsed_json_1, parsed_json_2

({'issue_type': 'other'}, {'issue_type': 'error'})

In [2]:
# BASE_MODEL_DIR = "/data/luomingkai/issue/models/Qwen/Qwen2.5-7B-Instruct"
# BASE_MODEL_DIR = "/root/autodl-fs/Qwen2.5-7B-Instruct"
BASE_MODEL_DIR = "/root/autodl-fs/DeepSeek-V2-Lite-Chat"

In [3]:
from transformers import AutoTokenizer
from vllm import LLM, SamplingParams

# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_DIR)

# Pass the default decoding hyperparameters of Qwen2.5-7B-Instruct
# max_tokens is for the maximum length for generation.
sampling_params = SamplingParams(temperature=0.5, top_p=1.0, repetition_penalty=1.05, max_tokens=512)

# Input the model name or path. Can be GPTQ or AWQ models.
model = LLM(model=BASE_MODEL_DIR, tensor_parallel_size=4, trust_remote_code=True)

# Prepare your prompts
prompt = "Tell me something about large language models."
messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

# generate outputs
outputs = model.generate([text], sampling_params)

# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")


INFO 12-27 22:49:28 config.py:112] Replacing legacy 'type' key with 'rope_type'
INFO 12-27 22:49:34 config.py:1020] Defaulting to use mp for distributed inference
WARNING 12-27 22:49:34 arg_utils.py:1013] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 12-27 22:49:34 config.py:1136] Chunked prefill is enabled with max_num_batched_tokens=512.
INFO 12-27 22:49:34 llm_engine.py:249] Initializing an LLM engine (v0.6.4.post1) with config: model='/root/autodl-fs/DeepSeek-V2-Lite-Chat', speculative_config=None, tokenizer='/root/autodl-fs/DeepSeek-V2-Lite-Chat', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=163840, download_dir=None, load_format=LoadFormat.AUTO, t

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


(VllmWorkerProcess pid=3286) INFO 12-27 22:50:39 model_runner.py:1077] Loading model weights took 7.3840 GB
(VllmWorkerProcess pid=3287) INFO 12-27 22:50:39 model_runner.py:1077] Loading model weights took 7.3840 GB
INFO 12-27 22:50:40 model_runner.py:1077] Loading model weights took 7.3840 GB
(VllmWorkerProcess pid=3289) INFO 12-27 22:50:39 model_runner.py:1077] Loading model weights took 7.3840 GB
WARNING 12-27 22:50:40 fused_moe.py:324] Using default MoE config. Performance might be sub-optimal! Config file not found at /root/miniconda3/lib/python3.12/site-packages/vllm/model_executor/layers/fused_moe/configs/E=64,N=352,device_name=NVIDIA_vGPU-32GB.json
(VllmWorkerProcess pid=3286) (VllmWorkerProcess pid=3289) (VllmWorkerProcess pid=3287) WARNING 12-27 22:50:40 fused_moe.py:324] Using default MoE config. Performance might be sub-optimal! Config file not found at /root/miniconda3/lib/python3.12/site-packages/vllm/model_executor/layers/fused_moe/configs/E=64,N=352,device_name=NVIDIA_v

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  2.18it/s, est. speed input: 59.18 toks/s, output: 124.92 toks/s]

Prompt: '<｜begin▁of▁sentence｜>You are a pirate chatbot who always responds in pirate speak!\n\nUser: Who are you?\n\nAssistant:', Generated text: " Arrr, I be the scallywag ye be lookin' for, matey! Ye can call me Captain Chat-a-lot, but mind ye, I be runnin' the high seas of knowledge and mischief here to keep ye entertained and informed, arrr!"


(VllmWorkerProcess pid=3286) (VllmWorkerProcess pid=3287) INFO 12-28 00:14:57 multiproc_worker_utils.py:240] Worker exiting
INFO 12-28 00:14:57 multiproc_worker_utils.py:240] Worker exiting
(VllmWorkerProcess pid=3289) INFO 12-28 00:14:57 multiproc_worker_utils.py:240] Worker exiting


In [4]:
def get_deepseek_output(
    model,
    tokenizer,
    messages_list,
    max_input_length=4096,
    max_tokens=512,
):
    text_list = []
    for messages in messages_list:
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        text_list.append(text)
        
    # sampling_params = SamplingParams(temperature=0.7, top_p=0.8, repetition_penalty=1.05, max_tokens=512)
    # sampling_params = SamplingParams(temperature=0.5, top_p=1.0, repetition_penalty=1.05, max_tokens=512)
    sampling_params = SamplingParams(temperature=0.3, top_p=1.0, repetition_penalty=1.05, max_tokens=max_tokens)
    

    outputs = model.generate(text_list, sampling_params)
    
    # Print the outputs.
    responses = []
    for output in outputs:
        prompt = output.prompt
        generated_text = output.outputs[0].text
        # print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")
        responses.append(generated_text)

    return responses

messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]
get_deepseek_output(model, tokenizer, [messages])

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  3.60it/s, est. speed input: 97.95 toks/s, output: 105.20 toks/s]


[' Arrr, matey! I be the fearsome Pirate Chatbot, aye! What be yer business on this fine day at sea?']

In [5]:
# data_path = "/home/luomingkai/workspace/issue_llm/issue_classify/issue_with_comments_framework/matched_results_test.json"
data_path = "/root/issue_classify/issue_with_comments_framework/matched_results_test_modify_other_update.json"
with open(data_path, encoding="utf-8") as fp:
    issue_data = json.load(fp)
    for idx, data in enumerate(issue_data):
        # print(issue_data)
        uesr, title, body, label, author_association = data["user"]["login"], data["title"], data["body"], data["tag_labels"], data["author_association"]
        comment_list = data["comments_list"]
        if len(comment_list) > 0:
            comment_list = [(com["user"]["login"], com["author_association"], com["body"]) for com in comment_list]

        print(f"uesr: {uesr}")
        print(f"Title: {title}")
        print(f"Body: {body}")
        print(f"label: {label}")
        print(f"author_association: {author_association}")
        print(f"comment_list: {comment_list}")
        
        
        # print()
        if idx > 2:
            break
        # print(title, description, label)


uesr: RainBoltz
Title: No output displayed or it gets stuck - OpenCV issue
Body: my terminal isn't responding after executed the command below: `./build/examples/openpose/rtpose.bin --imagelevel 1 --netcaffeopenpose.sh` , which was provided by official
label: deployment
author_association: NONE
comment_list: [('bigmoumou', 'NONE', 'I have the same problem too\r\n\r\nIs there any solution ?\r\n\r\n![help](https://cloud.githubusercontent.com/assets/16252975/25697753/cddcfe9e-30ee-11e7-82a2-2af97eb8460b.png)\r\n\r\n'), ('Shawnroom', 'NONE', 'I have the same issue, and wonder if there is any solution.'), ('gineshidalgo99', 'MEMBER', 'Sorry to hear that, we are working on fixing that error. We think it is due to OpenCV compiled with Qt or different visualization support.\r\n\r\nMeanwhile, you can make it work by:\r\n1. Completely uninstalling your current OpenCV version.\r\n2. Installing the default OpenCV from the Ubuntu repository: `apt-get install libopencv-dev`, or alternatively compili

In [6]:
global_issue_prompt = r"""
### **Role**  
You are an expert in GitHub repository analysis. Your task is to classify a given GitHub Issue into one of the following categories: **error**, **performance**, **deployment**, **question**, or **other**.

Analyze the conversation context thoroughly and determine the correct classification. If the issue is unrelated to the repository’s functionality or purpose, it must be categorized as **other**.

### **Issue Categories**  

- **error**:  
  Problems directly caused by the repository’s code, configuration, or inherent incompatibilities within the repository. For example:
  - Runtime errors.
  - Exceptions.
  - Failures in repository code execution.  

- **performance**:  
  Issues where the repository’s code or configuration leads to:
  - Slow execution times.
  - Resource bottlenecks (e.g., CPU, GPU, memory, or storage).
  - Inefficient resource usage (e.g., excessive memory or storage consumption).

- **deployment**:  
  Issues arising during the installation or deployment process that are specifically caused by:
  - Defects in the repository code.
  - Incomplete or inadequate documentation.
  - Configuration problems stemming from the repository.  

  *Note*: If the issue is caused by user errors (e.g., outdated dependencies, incorrect tools) or hardware/environmental limitations, it should be categorized as **question**.

- **question**:  
  Issues originating from:
  - Misunderstandings or failure to follow documentation.
  - Incorrect usage of the repository’s features.
  - Local environment misconfigurations not caused by the repository code or documentation.  

  *Note*: This category also includes:
  - User questions about usage scenarios.
  - Discussions seeking clarification or additional guidance.

- **other**:  
  Issues outside the predefined categories, including:
  1. Topics unrelated to the repository’s functionality.
  2. Feature requests or discussions beyond the repository’s purpose.
  3. Suggestions for improving documentation, usability, or community processes.


### **Process**  
**The process must be strictly executed, and the output must adhere to the defined JSON format.**

1. **Analyze the Conversation**:  
   Review the entire conversation context and evaluate the issue based on the evidence provided.

2. **Determine the Final Classification**:  
   Assign the issue to one of the five categories:
   - **error**
   - **performance**
   - **deployment**
   - **question**
   - **other**

3. **Output**:  
   Generate a single JSON object as the result, formatted as follows:  
   ```json
   {"issue_type": "<error|performance|deployment|question|other>"}
   ```

### **Examples**  

#### Example 1: Error
*Conversation:*  
- "Running model.fit() raises a KeyError related to missing labels in the dataset."  
- "We’ll patch data_loader.py to handle missing labels."  
- "The fix is merged. Let us know if it resolves the problem."

*Final Output:*  
```json
{"issue_type": "error"}
```

#### Example 2: Question
*Conversation:*  
- "I tried running the code, but the output looks weird. Is this a bug?"  
- "Have you verified if the input data matches the format described in the README?"  
- "I missed the formatting instructions. After fixing the input, it works fine."

*Final Output:*  
```json
{"issue_type": "question"}
```

#### Example 3: Performance
*Conversation:*  
- "The code runs much slower than expected for large datasets. Is there a way to optimize?"  
- "We could optimize the data processing step using multi-threading."

*Final Output:*  
```json
{"issue_type": "performance"}
```  

#### Example 4: Deployment
*Conversation:*  
- "The installation instructions don't mention the CUDA version required. The code fails on my setup."  
- "We’ll update the documentation to specify the supported CUDA versions."

*Final Output:*  
```json
{"issue_type": "deployment"}
```  

#### Example 5: Other
*Conversation:*  
- "It would be great if this repository supported visualization tools for monitoring model training."  
- "Thanks for the suggestion. We’ll consider this for future updates."

*Final Output:*  
```json
{"issue_type": "other"}
```  
"""

# 仅使用第一条issue的模版
question_prompt = """
````
*Conversation*:
- {}: ### Title: "{}" ### Body: "{}"
````
"""

# 使用comment的版本
question_comment_prompt = """
*Conversation*:
- "{} {}"
"""

comment_prompt = """
- "{}"
"""

ROLE_MAP_BEGIN = {
    "NONE": "ISSUE RAISER",
    "MEMBER": "MEMBER",
    "COLLABORATOR": "COLLABORATOR",
    "CONTRIBUTOR": "CONTRIBUTOR",
    "OWNER": "OWNER"
}

ROLE_MAP_COMMENT = {
    "NONE": "Commenter",
    "MEMBER": "MEMBER",
    "COLLABORATOR": "COLLABORATOR",
    "CONTRIBUTOR": "CONTRIBUTOR",
    "OWNER": "OWNER"
}

In [7]:
REPO_DESC = {
    "CMU-Perceptual-Computing-Lab/openpose": {
        "description": "OpenPose is a real-time multi-person keypoint detection library developed by the Carnegie Mellon Perceptual Computing Lab. It estimates human body, face, hands, and foot keypoints from single images, providing 2D real-time multi-person keypoint detection, including 15, 18, or 25-keypoint body/foot keypoint estimation, 2x21-keypoint hand keypoint estimation, and 70-keypoint face keypoint estimation. Additionally, it offers 3D real-time single-person keypoint detection and a calibration toolbox. OpenPose is compatible with various operating systems, including Ubuntu, Windows, and macOS, and supports CUDA (Nvidia GPU), OpenCL (AMD GPU), and CPU-only versions.",
        "url": "https://github.com/CMU-Perceptual-Computing-Lab/openpose"
    },
    "CorentinJ/Real-Time-Voice-Cloning": {
        "description": "Real-Time Voice Cloning is a Python-based tool that enables the cloning of voices in real-time. It utilizes deep learning models to synthesize speech that mimics a target voice, requiring only a few seconds of audio from the desired speaker. The repository provides code and instructions to train and use the voice cloning system.",
        "url": "https://github.com/CorentinJ/Real-Time-Voice-Cloning"
    },
    "JaidedAI/EasyOCR": {
        "description": "EasyOCR is an open-source Optical Character Recognition (OCR) library that supports over 80 languages. It is designed to be easy to use and provides accurate text recognition from images. The library is built on PyTorch and offers pre-trained models for various languages and scripts.",
        "url": "https://github.com/JaidedAI/EasyOCR"
    },
    "deepfakes/faceswap": {
        "description": "Faceswap is a deep learning-based tool for face-swapping in images and videos. It allows users to train models to swap faces between different subjects, providing a platform for experimenting with deepfake technology. The repository includes code for training and using the face-swapping models.",
        "url": "https://github.com/deepfakes/faceswap"
    },
    "deezer/spleeter": {
        "description": "Spleeter is an open-source tool developed by Deezer for source separation in music tracks. It uses deep learning models to separate audio into stems, such as vocals and accompaniment, enabling applications like karaoke and remixing. The repository provides pre-trained models and code for audio source separation.",
        "url": "https://github.com/deezer/spleeter"
    },
    "dusty-nv/jetson-inference": {
        "description": "Jetson Inference is a collection of deep learning inference samples and models for NVIDIA Jetson devices. It includes code for image classification, object detection, and segmentation, optimized for Jetson hardware. The repository provides pre-trained models and examples to demonstrate the capabilities of Jetson devices in AI applications.",
        "url": "https://github.com/dusty-nv/jetson-inference"
    },
    "iperov/DeepFaceLab": {
        "description": "DeepFaceLab is a deep learning tool for creating deepfakes, focusing on face-swapping in videos. It provides a comprehensive set of tools for training and applying deep learning models to perform face-swapping tasks. The repository includes code for data preparation, model training, and face-swapping applications.",
        "url": "https://github.com/iperov/DeepFaceLab"
    },
    "junyanz/pytorch-CycleGAN-and-pix2pix": {
        "description": "This repository provides PyTorch implementations of CycleGAN and pix2pix, two popular models for image-to-image translation tasks. CycleGAN enables image translation without paired examples, while pix2pix requires paired images for training. The repository includes code and pre-trained models for various image translation tasks.",
        "url": "https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix"
    },
    "mozilla/TTS": {
        "description": "Mozilla TTS is an open-source Text-to-Speech (TTS) engine that aims to make speech synthesis more accessible. It provides implementations of state-of-the-art TTS models, including Tacotron and FastSpeech, and supports training on custom datasets. The repository includes code for training and using TTS models.",
        "url": "https://github.com/mozilla/TTS"
    },
    "streamlit/streamlit": {
        "description": "Streamlit is an open-source app framework for Machine Learning and Data Science projects. It allows users to create interactive web applications for data analysis and visualization with minimal code. The repository provides the core framework and examples for building Streamlit applications.",
        "url": "https://github.com/streamlit/streamlit"
    },
    "microsoft/recommenders": {
        "description": "Recommenders is a project under the Linux Foundation of AI and Data. This repository contains examples and best practices for building recommendation systems, provided as Jupyter notebooks. The examples detail learnings on five key tasks: preparing and loading data for each recommendation algorithm, building models using various classical and deep learning recommendation algorithms such as Alternating Least Squares (ALS) or eXtreme Deep Factorization Machines (xDeepFM), evaluating algorithms with offline metrics, tuning and optimizing hyperparameters for recommendation models, and operationalizing models in a production environment on Azure. Several utilities are provided to support common tasks such as loading datasets in the format expected by different algorithms, evaluating model outputs, and splitting training/test data. Implementations of several state-of-the-art algorithms are included for self-study and customization in your own applications.",
        "url": "https://github.com/recommenders-team/recommenders"
    }
}

In [8]:
import pandas as pd
from sklearn.metrics import classification_report

TIMES = 10
experiments = []
# 保存所有实验结果的列表
all_reports = []

data_path = "/root/issue_classify/issue_with_comments_framework/matched_results_test_modify_other_update.json"
result_path = "/root/autodl-fs/log/deepseek_v2_lite_chat_prompt.xlsx"

for t in range(TIMES):
    origin_labels = []
    pred_labels = []
    
    join_index = []
    with open(data_path, encoding="utf-8") as fp:
        issue_data = json.load(fp)
        taged_data = issue_data
        all_messages = []
        for idx, data in enumerate(issue_data):
            raise_user, title, body, label, author_association = data["user"]["login"], data["title"], data["body"], data["tag_labels"], data["author_association"]
    
            url = data["html_url"]
            match = re.search(r"github\.com/([^/]+/[^/]+)", url)
            repository = match.group(1)
            repo_desc = REPO_DESC[repository]["description"]
            
            join_index.append(idx)
            
            comment_list = data["comments_list"]
            if len(comment_list) > 0:
                comment_list = [(com["user"]["login"], com["author_association"], com["body"]) for com in comment_list]
    
            messages = [
                {
                    "role": "system",
                    "content": global_issue_prompt
                },
                {
                    "role": "user",
                    "content": question_comment_prompt.format(
                        title, 
                        body
                        )
                }
            ]
            
            origin_labels.append(label)
    
            for user, author_association, body in comment_list:
                messages.append(
                    {
                        "role": "user",
                        "content": comment_prompt.format(
                            body
                            )
                    }
                )
                    
            all_messages.append(messages)
        
        responses = get_deepseek_output(model, tokenizer, all_messages, max_tokens=4096)
        for idx, response in enumerate(responses):
            label = origin_labels[idx]
            cls_result = extract_json_from_string(response)
            
            # print(f"label: {label}")
            # print(f"response: {response}")
            # print(f"cls_result: {cls_result}")
    
            if cls_result.get("issue_type") and isinstance(cls_result.get("issue_type"), str):
                cls_result = cls_result.get("issue_type")
                pred_labels.append(cls_result)
            else:
                pred_labels.append("other")

    # from collections import Counter
    # print(Counter(pred_labels))
    origin_labels = [x.lower() for x in origin_labels]
    pred_labels = [x.lower() for x in pred_labels]
    tmp_pred_labels = []
    for x in pred_labels:
        if x not in ["error", "performance", "deployment", "question", "other"]:
            x = 'other'
        tmp_pred_labels.append(x)
    pred_labels = tmp_pred_labels
    
    from sklearn.metrics import classification_report
    
    labels = ["error", "performance", "deployment", "question", "other"]
    label_map = {
        "error": 0, 
        "performance": 1,
        "deployment": 2,
        "question": 3,
        "other": 4
    }
    final_origin_labels_num = [label_map[l] for l in origin_labels]
    final_pred_labels_num = [label_map[l] for l in pred_labels]
    
    report = classification_report(final_origin_labels_num, final_pred_labels_num, target_names=labels)
    print(report)
    report_dict = classification_report(final_origin_labels_num, final_pred_labels_num, target_names=labels, output_dict=True)
    # 将字典转为 DataFrame
    df_report = pd.DataFrame(report_dict).transpose()
    df_report["experiment_id"] = t + 1  # 添加实验编号
    df_report.index.name = "category"
    df_report.reset_index(inplace=True)
    
    all_reports.append(df_report)
    all_reports.append(pd.DataFrame({"category": [""]}))  # 添加空行

final_df = pd.concat(all_reports, ignore_index=True)
final_df.to_excel(result_path, index=False)

Token indices sequence length is longer than the specified maximum sequence length for this model (34488 > 16384). Running this sequence through the model will result in indexing errors
Processed prompts: 100%|██████████| 1933/1933 [08:00<00:00,  4.02it/s, est. speed input: 7032.73 toks/s, output: 301.13 toks/s]


              precision    recall  f1-score   support

       error       0.39      0.52      0.44       251
 performance       0.22      0.25      0.23        60
  deployment       0.25      0.17      0.20       165
    question       0.64      0.51      0.57       921
       other       0.40      0.50      0.44       536

    accuracy                           0.47      1933
   macro avg       0.38      0.39      0.38      1933
weighted avg       0.49      0.47      0.47      1933



Processed prompts: 100%|██████████| 1933/1933 [08:00<00:00,  4.02it/s, est. speed input: 7032.39 toks/s, output: 310.21 toks/s]


              precision    recall  f1-score   support

       error       0.37      0.49      0.42       251
 performance       0.28      0.38      0.32        60
  deployment       0.26      0.19      0.22       165
    question       0.63      0.50      0.56       921
       other       0.39      0.49      0.44       536

    accuracy                           0.47      1933
   macro avg       0.39      0.41      0.39      1933
weighted avg       0.49      0.47      0.47      1933



Processed prompts: 100%|██████████| 1933/1933 [08:04<00:00,  3.99it/s, est. speed input: 6971.48 toks/s, output: 294.36 toks/s]


...

              precision    recall  f1-score   support

       error       0.38      0.53      0.44       251
 performance       0.21      0.22      0.21        60
  deployment       0.27      0.19      0.23       165
    question       0.65      0.50      0.56       921
       other       0.40      0.52      0.45       536

    accuracy                           0.47      1933
   macro avg       0.38      0.39      0.38      1933
weighted avg       0.50      0.47      0.48      1933



Processed prompts: 100%|██████████| 1933/1933 [07:49<00:00,  4.12it/s, est. speed input: 7200.64 toks/s, output: 311.20 toks/s]


              precision    recall  f1-score   support

       error       0.37      0.51      0.43       251
 performance       0.23      0.27      0.25        60
  deployment       0.24      0.19      0.21       165
    question       0.63      0.49      0.56       921
       other       0.40      0.50      0.44       536

    accuracy                           0.46      1933
   macro avg       0.37      0.39      0.38      1933
weighted avg       0.49      0.46      0.47      1933



Processed prompts: 100%|██████████| 1933/1933 [07:53<00:00,  4.08it/s, est. speed input: 7137.91 toks/s, output: 308.84 toks/s]

              precision    recall  f1-score   support

       error       0.38      0.54      0.45       251
 performance       0.20      0.23      0.22        60
  deployment       0.27      0.21      0.23       165
    question       0.63      0.48      0.55       921
       other       0.40      0.50      0.44       536

    accuracy                           0.47      1933
   macro avg       0.38      0.39      0.38      1933
weighted avg       0.49      0.47      0.47      1933



In [9]:
import gc
import torch

# 假设你的 vllm 模型对象是 `model`
del model  # 删除模型对象
torch.cuda.empty_cache()  # 清空 GPU 的缓存
gc.collect()  # 强制进行垃圾回收

import sys
# 删除 `vllm` 和相关依赖
del sys.modules["vllm"]
torch.cuda.empty_cache()
gc.collect()

INFO 12-28 00:14:57 multiproc_worker_utils.py:133] Terminating local vLLM worker processes


0

In [10]:
# all question类别打标之后的
from collections import Counter
print(Counter(pred_labels))
origin_labels = [x.lower() for x in origin_labels]
pred_labels = [x.lower() for x in pred_labels]
tmp_pred_labels = []
for x in pred_labels:
    if x not in ["error", "performance", "deployment", "question", "other"]:
        x = 'other'
    tmp_pred_labels.append(x)
pred_labels = tmp_pred_labels

from sklearn.metrics import classification_report

labels = ["error", "performance", "deployment", "question", "other"]
label_map = {
    "error": 0, 
    "performance": 1,
    "deployment": 2,
    "question": 3,
    "other": 4
}
final_origin_labels_num = [label_map[l] for l in origin_labels]
final_pred_labels_num = [label_map[l] for l in pred_labels]

report = classification_report(final_origin_labels_num, final_pred_labels_num, target_names=labels)
print(report)

Counter({'other': 725, 'question': 645, 'error': 308, 'deployment': 174, 'performance': 70, 'compilation/installation error': 3, 'performance,deployment': 2, 'feature-request': 2, 'development': 1, 'feature_request': 1, 'work-in-progress': 1, 'development/enhancement': 1})
              precision    recall  f1-score   support

       error       0.39      0.48      0.43       251
 performance       0.24      0.28      0.26        60
  deployment       0.29      0.30      0.29       165
    question       0.64      0.45      0.53       921
       other       0.40      0.55      0.46       536

    accuracy                           0.46      1933
   macro avg       0.39      0.41      0.40      1933
weighted avg       0.50      0.46      0.47      1933

